## Projeto Final do Módulo de Lógica de Programação II
#### Descrição:
    Implementar o algoritmo K-Nearest Neighbors (KNN) utilizando o conhecimento adquirido no curso até o momento.
### Regras:
- Se você precisar de uma função para computar algo crie ela
- Não é permitido usar nenhum módulo externo como numpy e math
- Use apenas os objetos e fluxos visto até o momento no curso

#### Grupo composto por:
- Rayssa Vilaça



### Funções

In [91]:
from functools import reduce


def euclidean_distance(x1, x2):
    
    """
    Calcula a distância euclidiana entre dois pontos no espaço.

    Esta função recebe dois pontos, 'x1' e 'x2', e retorna a distância euclidiana entre eles.

    Parâmetros:
    x1 (array-like): Coordenadas do primeiro ponto.
    x2 (array-like): Coordenadas do segundo ponto.

    Retorna:
    float: A distância euclidiana entre os pontos 'x1' e 'x2'.
    """
    
    squared_diff = [(x1[i] - x2[i]) ** 2 for i in range(len(x1))]
    total_sum = reduce(lambda x, y: x + y, squared_diff)
    return total_sum ** (1/2)

In [92]:
def accuracy_score(y_test, y_pred):
    """
    Calcula a acurácia de um modelo de classificação.

    Esta função compara os rótulos verdadeiros ('y_test') com as previsões do modelo ('y_pred')
    e calcula a acurácia do modelo.

    Parâmetros:
    y_test (array-like): Rótulos verdadeiros.
    y_pred (array-like): Previsões do modelo.

    Retorna:
    float: A acurácia do modelo, variando de 0 a 1, onde 1 representa uma previsão perfeita.
    """
    
    data = zip(y_test, y_pred)
    same_label = list(filter(lambda x: x[0] == x[1], data))
    return len(same_label) / len(y_test)

In [93]:
import random

def train_test_split(X, y, test_size=0.3, random_state=None):
    
    """
    Divide um conjunto de dados em subconjuntos aleatórios de treinamento e teste.

    Esta função recebe um conjunto de características (X) e seus rótulos (y) e realiza a divisão em conjuntos
    de treinamento e teste. A divisão é realizada aleatoriamente, mantendo a proporção definida por 'test_size'.

    Parâmetros:
    X (array-like): Conjunto de características.
    y (array-like): Rótulos correspondentes às características.
    test_size (float, optional): A proporção do conjunto de teste. Padrão é 0.3.
    random_state (int, optional): Semente para a geração de números aleatórios. Padrão é 101.

    Retorna:
    tuple: Uma tupla contendo quatro elementos - X_train, X_test, y_train, y_test.
        X_train (array-like): Conjunto de características de treinamento.
        X_test (array-like): Conjunto de características de teste.
        y_train (array-like): Rótulos correspondentes às características de treinamento.
        y_test (array-like): Rótulos correspondentes às características de teste.
    """
    if random_state != None:
        random.seed(random_state)
    
    data = list(zip(X, y))
    random.shuffle(data)
    
    split_index = int(len(data) * (1 - test_size))
    
    X_train, y_train = zip(*data[:split_index])
    X_test, y_test = zip(*data[split_index:])
    
    return list(X_train), list(X_test), list(y_train), list(y_test)

### Classes

In [94]:

class Node:
    def __init__(self, value, axis, idx):
        self.value = value
        self.axis = axis
        self.idx = idx
        self.left = None
        self.right = None

In [95]:
class label_encoder():
    def __init__(self):
        self.labels_mapping = {}
        self.inverse_labels_mapping = {}
    
    def fit(self, labels):
        unique_labels = list(set(labels))
        
        self.labels_mapping = {unique_labels[idx]: idx for idx in range(len(unique_labels))}
        self.inverse_labels_mapping = {idx: unique_labels[idx] for idx in range(len(unique_labels))}
    
    def transform(self, labels):
        return [self.labels_mapping[label] for label in labels]
    
    def inverse_transform(self, encoded_labels):
        return [self.inverse_labels_mapping[encoded_label] for encoded_label in encoded_labels]
        

In [96]:
class kd_tree():
    def __init__(self, data):
        self.data = data
        self.root = None
    
    def _build(self, data, depth):
        if len(data) == 0:
            return None
    
        # if len(data) == 1:
        #     return Node(data[0])
        
        n_variables = len(data[0])
        n_data = len(data)
        
        axis = depth % n_variables
        
        sorted_data = sorted(data, key=lambda x: x[axis])
        median = n_data // 2
        
        node = Node(sorted_data[median], axis, median)
        node.left = self._build(sorted_data[:median], depth + 1)
        node.right = self._build(sorted_data[median+1:], depth + 1)
        
        return node
    
    def build(self):
        self.root = self._build(self.data, 0)
    
    def _neighbor_nearest(self, node, current_node, best_node=None, best_distance=99999999):
        if current_node == None:
            return best_node, best_distance
        
        distance = euclidean_distance(node, current_node.value)
        
        if distance < best_distance:
            best_node = current_node
            best_distance = distance
        
        if node[current_node.axis] < current_node.value[current_node.axis]:
            b_child = current_node.left
            w_child = current_node.right
        
        else:
            b_child = current_node.right
            w_child = current_node.left
            
        best_node, best_distance = self._neighbor_nearest(node, b_child, best_node, best_distance)
        
        if abs(node[current_node.axis] - current_node.value[current_node.axis]) < best_distance:
            best_node, best_distance = self._neighbor_nearest(node, w_child, best_node, best_distance)
        
        return best_node, best_distance

    def neighbor_nearest(self, node):
        return self._neighbor_nearest(node, self.root)
    
    def _print_tree(self, root):
    
        if root == None:
            return [], 0, 0, 0
        
        current_root = str(root.value[root.axis])
        size_root = distance = len(current_root)
        
        l_child, l_start, l_end, l_size = self._print_tree(root.left)
        r_child, r_start, r_end, r_size  = self._print_tree(root.right)
        
        linha1 = []
        linha2 = []
        
        if l_size > 0:
            l_median = (l_start + l_end) // 2 + 1
            linha1.append(" " * (l_median + 1))
            linha1.append("_" * (l_size - l_median))
            linha2.append(" " * l_median + "/")
            linha2.append(" " * (l_size - l_median))
            distance += 1
            start = l_size + 1
        else:
            start = 0
        
        end = start + size_root - 1
        
        linha1.append(current_root)
        linha2.append(" " * size_root)
        
        if r_size > 0:
            r_median = (r_start + r_end) // 2
            linha1.append("_" * r_median)
            linha1.append(" " * (r_size - r_median + 1))
            linha2.append(" " * r_median + "\\")
            linha2.append(" " * (r_size - r_median))
            distance += 1
            
            
        child = ["".join(linha1), "".join(linha2)]
        size = len(child[0])
        
        size_l_child = len(l_child)
        size_r_child = len(r_child)
        n = max(size_l_child, size_r_child)
        
        for idx in range(n):
            l = l_child[idx] if idx < size_l_child else " " * l_size
            r = r_child[idx] if idx < size_r_child else " " * r_size
            child.append(l + " " * distance + r)

        return child, start, end, size
    
    def __str__(self):
        lines = self._print_tree(self.root)[0]
        return "\n".join(lines)

In [97]:
class k_neighbors_classifier():
    
    def __init__(self):
        
        """
        Inicializa o classificador KNN.

        Parâmetros:
        n_neighbors (int): Número de vizinhos a serem considerados.
        """
    
    def fit(self, X_train, y_train):
        
        """
        Treina o modelo KNN com dados de treinamento.

        Parâmetros:
        X_train (list): Lista de dados de treinamento.
        y_train (list): Lista de rótulos de treinamento.
        """
        
        self.X_train = X_train
        self.y_train = y_train
        
        self.kd_tree = kd_tree(self.X_train)
        self.kd_tree.build()
        # print(self.kd_tree)
    
    def predict(self, X_test):
        
        """
        Faz previsões usando o modelo KNN.

        Parâmetros:
        X_test (list): Lista de dados de teste.
        
        Retorna:
        list: Lista de previsões para os dados de teste.
        """
        
        return [self._get_label(self.kd_tree.neighbor_nearest(x)[0]) for x in X_test]
    
    def _get_label(self, node):
        idx = node.idx
        return self.y_train[idx]
    

### Base de dados cadastro de clientes de uma empresa de investimentos

#### Descrição:
Os dados abaixo são referentes ao cadastro de clientes de uma empresa de investimentos com suas respectivas carteira de investimentos, que indica se esse cliente tem o perfil de investidor Conservador, Moderado ou Agressivo. O nosso intuito é, a partir do investimento de alguns clientes que já tem um perfil definido, estimar esse perfil para aqueles que ainda não estão classificado, afim de oferecer novos produtos que sejam mais adequados a eles.

Os dados abaixo seguem o seguinte padrão:

[CPF: INT, Perfil Do Investidor: STRING, Carteira de Investimento: TUPLA]

In [98]:
data = [[66707599984, 'Conservador', (5100., 3500., 1400., 200.)],
[55695397315, 'Conservador', (4900., 3000., 1400., 200.)],
[63743886918, 'Conservador', (4700., 3200., 1300., 200.)],
[55941368774, 'Conservador', (4600., 3100., 1500., 200.)],
[75486280874, 'Conservador', (5000., 3600., 1400., 200.)],
[53164949799, 'Conservador', (5400., 3900., 1700., 400.)],
[39898704131, 'Conservador', (4600., 3400., 1400., 300.)],
[53740901207, 'Conservador', (5000., 3400., 1500., 200.)],
[51735950236, 'Conservador', (4400., 2900., 1400., 200.)],
[47305108951, 'Conservador', (4900., 3100., 1500., 100.)],
[63858864633, 'Conservador', (5400., 3700., 1500., 200.)],
[53363167240, 'Conservador', (4800., 3400., 1600., 200.)],
[72133754195, 'Conservador', (4800., 3000., 1400., 100.)],
[52802483512, 'Conservador', (4300., 3000., 1100., 100.)],
[57925287214, 'Conservador', (4800., 3400., 1900., 200.)],
[74354632224, 'Conservador', (5000., 3000., 1600., 200.)],
[64020216626, 'Conservador', (5000., 3400., 1600., 400.)],
[78223722856, 'Conservador', (5200., 3500., 1500., 200.)],
[58245228846, 'Conservador', (5200., 3400., 1400., 200.)],
[74490686776, 'Conservador', (4700., 3200., 1600., 200.)],
[48646824781, 'Conservador', (4800., 3100., 1600., 200.)],
[77381458676, 'Conservador', (5400., 3400., 1500., 400.)],
[41615431874, 'Conservador', (5200., 4100., 1500., 100.)],
[52163844491, 'Conservador', (5500., 4200., 1400., 200.)],
[70276304567, 'Conservador', (4900., 3100., 1500., 200.)],
[69119828185, 'Conservador', (5000., 3200., 1200., 200.)],
[65441690046, 'Conservador', (5500., 3500., 1300., 200.)],
[56457227894, 'Conservador', (4900., 3600., 1400., 100.)],
[46939428126, 'Conservador', (4400., 3000., 1300., 200.)],
[60979942480, 'Conservador', (5100., 3400., 1500., 200.)],
[41648583220, 'Conservador', (5000., 3500., 1300., 300.)],
[50376331791, 'Conservador', (4500., 2300., 1300., 300.)],
[67008801023, 'Conservador', (4400., 3200., 1300., 200.)],
[72149193419, 'Conservador', (5000., 3500., 1600., 600.)],
[62830733382, 'Conservador', (5100., 3800., 1900., 400.)],
[56716675811, 'Conservador', (4800., 3000., 1400., 300.)],
[61089667146, 'Conservador', (5100., 3800., 1600., 200.)],
[47795509468, 'Conservador', (4600., 3200., 1400., 200.)],
[60899885693, 'Conservador', (5300., 3700., 1500., 200.)],
[53433670705, 'Conservador', (5000., 3300., 1400., 200.)],
[54850120580, 'Moderado', (7000., 3200., 4700., 1400.)],
[71457789994, 'Moderado', (6400., 3200., 4500., 1500.)],
[67692777563, 'Moderado', (6900., 3100., 4900., 1500.)],
[43133573182, 'Moderado', (5500., 2300., 4000., 1300.)],
[55150612815, 'Moderado', (6500., 2800., 4600., 1500.)],
[48211725243, 'Moderado', (5700., 2800., 4500., 1300.)],
[76686463776, 'Moderado', (6300., 3300., 4700., 1600.)],
[71971000560, 'Moderado', (4900., 2400., 3300., 1000.)],
[40307235992, 'Moderado', (6600., 2900., 4600., 1300.)],
[44826533081, 'Moderado', (5200., 2700., 3900., 1400.)],
[45735414894, 'Moderado', (5900., 3200., 4800., 1800.)],
[57137146514, 'Moderado', (6100., 2800., 4000., 1300.)],
[53657058251, 'Moderado', (6300., 2500., 4900., 1500.)],
[52941460485, 'Moderado', (6100., 2800., 4700., 1200.)],
[44306600683, 'Moderado', (6400., 2900., 4300., 1300.)],
[43460747924, 'Moderado', (6600., 3000., 4400., 1400.)],
[75590376075, 'Moderado', (6800., 2800., 4800., 1400.)],
[68267282206, 'Moderado', (6700., 3000., 5000., 1700.)],
[77567920298, 'Moderado', (6000., 2900., 4500., 1500.)],
[67600419504, 'Moderado', (5700., 2600., 3500., 1000.)],
[44902189811, 'Moderado', (5500., 2400., 3800., 1100.)],
[62966866614, 'Moderado', (5500., 2400., 3700., 1000.)],
[56182108880, 'Moderado', (5800., 2700., 3900., 1200.)],
[78299785392, 'Moderado', (6000., 2700., 5100., 1600.)],
[45206071878, 'Moderado', (5400., 3000., 4500., 1500.)],
[57381925887, 'Moderado', (6000., 3400., 4500., 1600.)],
[65654934891, 'Moderado', (6700., 3100., 4700., 1500.)],
[56130640481, 'Moderado', (6300., 2300., 4400., 1300.)],
[59667611672, 'Moderado', (5600., 3000., 4100., 1300.)],
[40349334385, 'Moderado', (5500., 2500., 4000., 1300.)],
[68422640081, 'Moderado', (5500., 2600., 4400., 1200.)],
[55245923439, 'Moderado', (6100., 3000., 4600., 1400.)],
[51286696873, 'Moderado', (5800., 2600., 4000., 1200.)],
[41065279767, 'Moderado', (5000., 2300., 3300., 1000.)],
[42866454119, 'Moderado', (5600., 2700., 4200., 1300.)],
[61962944542, 'Moderado', (5700., 3000., 4200., 1200.)],
[48623501235, 'Moderado', (5700., 2900., 4200., 1300.)],
[49475220139, 'Moderado', (6200., 2900., 4300., 1300.)],
[52245218531, 'Moderado', (5100., 2500., 3000., 1100.)],
[50932926697, 'Moderado', (5700., 2800., 4100., 1300.)],
[47432932248, 'Agressivo', (6300., 3300., 6000., 2500.)],
[39321991579, 'Agressivo', (5800., 2700., 5100., 1900.)],
[46283759608, 'Agressivo', (7100., 3000., 5900., 2100.)],
[56996272538, 'Agressivo', (6300., 2900., 5600., 1800.)],
[77232189978, 'Agressivo', (6500., 3000., 5800., 2200.)],
[77183282421, 'Agressivo', (7600., 3000., 6600., 2100.)],
[42857147573, 'Agressivo', (4900., 2500., 4500., 1700.)],
[39331584043, 'Agressivo', (7300., 2900., 6300., 1800.)],
[48130345228, 'Agressivo', (6700., 2500., 5800., 1800.)],
[71422443953, 'Agressivo', (7200., 3600., 6100., 2500.)],
[72508507904, 'Agressivo', (6900., 3200., 5700., 2300.)],
[41188727558, 'Agressivo', (5600., 2800., 4900., 2000.)],
[61358776640, 'Agressivo', (7700., 2800., 6700., 2000.)],
[66934042323, 'Agressivo', (6300., 2700., 4900., 1800.)],
[40622495567, 'Agressivo', (6700., 3300., 5700., 2100.)],
[57221661311, 'Agressivo', (7200., 3200., 6000., 1800.)],
[45159362930, 'Agressivo', (6200., 2800., 4800., 1800.)],
[45018975174, 'Agressivo', (6100., 3000., 4900., 1800.)],
[70685429140, 'Agressivo', (6400., 2800., 5600., 2100.)],
[61808723477, 'Agressivo', (7200., 3000., 5800., 1600.)],
[56363906548, 'Agressivo', (7400., 2800., 6100., 1900.)],
[39646194720, 'Agressivo', (7900., 3800., 6400., 2000.)],
[55385494438, 'Agressivo', (6400., 2800., 5600., 2200.)],
[75796138061, 'Agressivo', (6300., 2800., 5100., 1500.)],
[53595767857, 'Agressivo', (6100., 2600., 5600., 1400.)],
[48758828080, 'Agressivo', (7700., 3000., 6100., 2300.)],
[58387651356, 'Agressivo', (6300., 3400., 5600., 2400.)],
[72846931192, 'Agressivo', (6400., 3100., 5500., 1800.)],
[47046896346, 'Agressivo', (6000., 3000., 4800., 1800.)],
[69730292799, 'Agressivo', (6900., 3100., 5400., 2100.)],
[48177836349, 'Agressivo', (6700., 3100., 5600., 2400.)],
[57976326635, 'Agressivo', (6900., 3100., 5100., 2300.)],
[55710813002, 'Agressivo', (5800., 2700., 5100., 1900.)],
[64028580439, 'Agressivo', (6800., 3200., 5900., 2300.)],
[49962942971, 'Agressivo', (6700., 3300., 5700., 2500.)],
[47250893163, 'Agressivo', (6700., 3000., 5200., 2300.)],
[75559276274, 'Agressivo', (6300., 2500., 5000., 1900.)],
[58529878272, 'Agressivo', (6500., 3000., 5200., 2000.)],
[76005896622, 'Agressivo', (6200., 3400., 5400., 2300.)],
[49212614633, 'Agressivo', (5900., 3000., 5100., 1800.)]]

no_class = [[45926320819, '', (5800., 4000., 1200., 200.)],
[52559670741, '', (5700., 4400., 1500., 400.)],
[59016004832, '', (5400., 3900., 1300., 400.)],
[66175672425, '', (5100., 3500., 1400., 300.)],
[53330429526, '', (5700., 3800., 1700., 300.)],
[43765563403, '', (5100., 3800., 1500., 300.)],
[68020822591, '', (5400., 3400., 1700., 200.)],
[53939481689, '', (5100., 3700., 1500., 400.)],
[47014057561, '', (4600., 3600., 1000., 200.)],
[57183542047, '', (5100., 3300., 1700., 500.)],
    
[68518284363, '', (5000., 2000., 3500., 1000.)],
[65806049885, '', (5900., 3000., 4200., 1500.)],
[54128073086, '', (6000., 2200., 4000., 1000.)],
[41306785494, '', (6100., 2900., 4700., 1400.)],
[65234831039, '', (5600., 2900., 3600., 1300.)],
[50964498067, '', (6700., 3100., 4400., 1400.)],
[50810951429, '', (5600., 3000., 4500., 1500.)],
[48765044397, '', (5800., 2700., 4100., 1000.)],
[41960083761, '', (6200., 2200., 4500., 1500.)],
[76657763082, '', (5600., 2500., 3900., 1100.)],
    
[64726487742, '', (6500., 3200., 5100., 2000.)],
[75746566283, '', (6400., 2700., 5300., 1900.)],
[78576734793, '', (6800., 3000., 5500., 2100.)],
[56440141847, '', (5700., 2500., 5000., 2000.)],
[66827423000, '', (5800., 2800., 5100., 2400.)],
[45267873396, '', (6400., 3200., 5300., 2300.)],
[46387191493, '', (6500., 3000., 5500., 1800.)],
[54273611732, '', (7700., 3800., 6700., 2200.)],
[75135392881, '', (7700., 2600., 6900., 2300.)],
[64703873108, '', (6000., 2200., 5000., 1500.)]]

In [99]:
labels = [info[1] for info in data]

lencoder = label_encoder()
lencoder.fit(labels)

encoded_data = [[info[0], lencoder.transform([info[1]])[0], info[2]] for info in data]


In [100]:
X = [info[2] for info in encoded_data]
y = [info[1] for info in encoded_data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [101]:
modelo = k_neighbors_classifier()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

In [102]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo é de {accuracy * 100:.2f}%")

Acurácia do modelo é de 33.33%


### Base de dados diabetes
[Link para a base de dados](https://www.kaggle.com/datasets/rajakali/diabetesknn/data)

In [103]:
with open('assets/diabetes.csv', 'r') as arquivo_csv: 
    linhas = arquivo_csv.readlines()
    
    data = []
    
    for linha in linhas[1:]:
        campos = linha.strip().split(',')

        pregnancies = int(campos[0])
        glucose = int(campos[1])
        blood_pressure = int(campos[2])
        skin_thickness = int(campos[3])
        insulin = int(campos[4])
        bmi = float(campos[5])
        diabetes_pedigree_function = float(campos[6])
        age = int(campos[7])
        outcome = int(campos[8])

        linha_processada = [pregnancies,
                            glucose,
                            blood_pressure,
                            skin_thickness,
                            insulin,
                            bmi, 
                            diabetes_pedigree_function, 
                            age, 
                            outcome]
        
        data.append(linha_processada) 

In [104]:
X = [info[:-1] for info in data]
y = [info[-1] for info in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [105]:
modelo = k_neighbors_classifier()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

In [106]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo é de {accuracy * 100:.2f}%")

Acurácia do modelo é de 65.37%


### Matriz de Confusão

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*P7ZXAWS1QJl9k7h1' width=50%>

In [107]:
def confusion_matrix(y_true, y_pred):
    tp, fp, tn, fn = 0, 0, 0, 0
    
    for true, pred in zip(y_true, y_pred):
        if true == 1 and pred == 1:
            tp += 1
        elif true == 0 and pred == 1:
            fp += 1
        elif true == 0 and pred == 0:
            tn += 1
        elif true == 1 and pred == 0:
            fn += 1
    
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    negative_predictive_value = tn / (tn + fn)
    
    print(f"{'Matriz de Confusão':^50}")
    print()
    print(f"{'True v Pred >':<15}|{'Positivo':^10}|{'Negativo':^10}|")
    print("-" * 50)
    print(f"{'Positivo':<15}|{tp:^10}|{fn:^10}|{sensitivity:^10.2f}")
    print("-" * 50)
    print(f"{'Negativo':<15}|{fp:^10}|{tn:^10}|{specificity:^10.2f}")
    print("-" * 50)
    print(f"{' ':<15}|{precision:^10.2f}|{negative_predictive_value:^10.2f}|{accuracy:^10.2f}")

In [108]:
confusion_matrix(y_test, y_pred)

                Matriz de Confusão                

True v Pred >  | Positivo | Negativo |
--------------------------------------------------
Positivo       |    3     |    72    |   0.04   
--------------------------------------------------
Negativo       |    8     |   148    |   0.95   
--------------------------------------------------
               |   0.27   |   0.67   |   0.65   
